reference : https://dacon.io/competitions/official/236093/codeshare/8182?page=1&dtype=recent

In [8]:
import h2o
import pandas as pd

In [1]:
h2o.init()

train = h2o.import_file('train.csv')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.19" 2023-04-18; OpenJDK Runtime Environment (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1); OpenJDK 64-Bit Server VM (build 11.0.19+7-post-Ubuntu-0ubuntu120.04.1, mixed mode, sharing)
  Starting server from /home/jjaegii/dacon/venv/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpv1e59gc3
  JVM stdout: /tmp/tmpv1e59gc3/h2o_jjaegii_started_from_python.out
  JVM stderr: /tmp/tmpv1e59gc3/h2o_jjaegii_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Asia/Seoul
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.40.0.4
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_jjaegii_3ounxn
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.861 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [2]:
x = train.columns
y = "가격"
x.remove(y)

In [3]:
from h2o.automl import H2OAutoML

aml = H2OAutoML(
    max_models=10,
    seed=42,
    # max_runtime_secs=360,
    sort_metric='MAE'
)

aml.train(
    x=x,
    y=y,
    training_frame=train
)

leaderboard = aml.leaderboard
print(leaderboard.head())

AutoML progress: |
02:00:07.524: _train param, Dropping bad and constant columns: [ID]

██
02:00:42.113: _train param, Dropping bad and constant columns: [ID]

██
02:00:56.126: _train param, Dropping bad and constant columns: [ID]

█
02:01:34.769: _train param, Dropping bad and constant columns: [ID]

███
02:02:03.536: _train param, Dropping bad and constant columns: [ID]

██████████████████████████████████████
02:10:04.781: _train param, Dropping bad and constant columns: [ID]


02:10:17.744: _train param, Dropping bad and constant columns: [ID]

█
02:10:33.4: _train param, Dropping bad and constant columns: [ID]


02:10:57.421: _train param, Dropping bad and constant columns: [ID]

██
02:11:39.511: _train param, Dropping bad and constant columns: [ID]

████████
02:18:45.186: _train param, Dropping unused columns: [ID]

█
02:18:47.245: _train param, Dropping unused columns: [ID]

█████| (done) 100%
model_id                                                    mae     rmse      mse      

현재 가장 낮은 MAE 값은 5.83384

In [4]:
model = aml.leader

In [5]:
test = h2o.import_file('test.csv')

pred = model.predict(test)

stackedensemble prediction progress: |

███████████████████████████████████████████| (done) 100%


/home/jjaegii/dacon/venv/lib/python3.8/site-packages/h2o/job.py:83: UserWarning: Test/Validation dataset column '판매도시' has levels not trained on: ["AAI", "AGG", "AJB", "AJM", "AXK", "AZD", "BJC", "BOI", "BRV", "BSR", ...280 not listed..., "ZLI", "ZNE", "ZOH", "ZPO", "ZRL", "ZTU", "ZVM", "ZXL", "ZYG", "ZZF"]
  warnings.warn(w)


In [9]:
pred_df = pd.DataFrame(pred.as_data_frame())

In [11]:
import pandas as pd

submission = pd.read_csv('sample_submission.csv')
submission['가격'] = pred_df
submission.to_csv('H2O_submit.csv', index = False)